In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from dataset import LandscapeSimulationDataset
from model import PhiNN
from model_training import train_model
from helpers import select_device, jump_function, mean_cov_loss, kl_divergence_est

In [ ]:
outdir = "out/testing"
dtype = torch.float64
device = 'cpu'

In [ ]:
datdir_train = "data/model_training_data_2"
datdir_valid = "data/model_validation_data_2"

nsims_train = 10
nsims_valid = 5
ndims = 2
nsigs = 2
ncells = 100
dt = 1e-1
sigma = 1e-2

batch_size = 4

In [ ]:
train_dataset = LandscapeSimulationDataset(
    datdir_train, nsims_train, ndims, 
    transform='tensor', 
    target_transform='tensor',
    dtype=dtype,
)

validation_dataset = LandscapeSimulationDataset(
    datdir_valid, nsims_valid, ndims, 
    transform='tensor', 
    target_transform='tensor',
    dtype=dtype,
)

train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)

validation_dataloader = DataLoader(
    validation_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

print("Training Dataset Length:", len(train_dataset))
print("Validation Dataset Length:", len(validation_dataset))
print("Training DataLoader Length:", len(train_dataloader))
print("Validation DataLoader Length:", len(validation_dataloader))

In [ ]:
# Construct the model
f_signal = lambda t, p: jump_function(t, p[...,0], p[...,1:3], p[...,3:])
model = PhiNN(
    ndim=ndims, nsig=nsigs, 
    f_signal=f_signal, nsigparams=5,
    ncells=ncells, 
    infer_noise=True,
    sigma=sigma,
    device=device,
    dtype=dtype,
    sample_cells=True,
).to(device)

# loss_fn = mean_cov_loss
loss_fn = kl_divergence_est

learning_rate = 1e-2
momentum = 0.9

optimizer = torch.optim.SGD(
    model.parameters(), 
    lr=learning_rate, 
    momentum=momentum
)

In [ ]:
train_model(
    model, dt, loss_fn, optimizer, 
    train_dataloader, validation_dataloader,
    num_epochs=2,
    batch_size=batch_size,
    device=device,
    dtype=dtype,
    model_name='testmodel',
    outdir=outdir,
)